# Demo notebook for VERB-3D Model Reader

In [ ]:
# Delete the time files before running.
model = 'VERB-3D'
variables_requested = ['PSD_lea', 'L', 'E_e', 'alpha_e', 'flux_lea', 'PSD_lmk']  

# change file path to where data is stored on your machine
#verb_dataset = "C:\\work\\Codes\\CCMC\\simulations_CCMC\\Alexander_Drozdov_031221_IM_2\\Output\\"
# verb_dataset = "C:\\work\\Codes\\CCMC\\simulations_CCMC\\Alexander_Drozdov_031221_IM_5\\Output\\"
# verb_dataset = "C:\\work\\Codes\\CCMC\\simulations_CCMC\\Alexander_Drozdov_031221_IM_6\\Output\\"
verb_dataset = "C:\\work\\Codes\\CCMC\\simulations_CCMC\\Alexander_Drozdov_060322_IM_4\\Output\\"


Next section can be used to download the dataset from CCMC. Set `DOWNLOAD_TEST_DATASET` to `True` and change `ccmc_download_path` accordingly.

In [ ]:
import os
import requests

# Flag to download tests dataset
DOWNLOAD_TEST_DATASET = False

# Path where download VERB code output
ccmc_download_path = "C:\\work\\Codes\\CCMC\\simulations_CCMC\\"
# Name of the simulation folder
ccmc_simulation_name = "Alexander_Drozdov_060322_IM_4"
# CCMC url
ccmc_ror_url = 'https://ccmc.gsfc.nasa.gov/results/api/get_run_file.php'

# Download process
filenames = ["Output/OutPSD.dat", "Output/out1d.dat", "Output/perp_grid.plt", "ror_metadata.json"]

if DOWNLOAD_TEST_DATASET:
    # Create the folder for the runnumber and the Output subfolder
    folder_path = os.path.join(ccmc_download_path, ccmc_simulation_name, 'Output')
    os.makedirs(folder_path, exist_ok=True)
    
    # Download and save files in a structured way
    for filename in filenames:    
        file_path = os.path.join(folder_path, filename.split('/')[-1])
        if 'json' not in filename:
            file_path = os.path.join(folder_path, filename.split('/')[-1])
        
        url = f'{ccmc_ror_url}?runnumber={ccmc_simulation_name}&filename={filename}'
        
        # Download and save the file
        response = requests.get(url)
        with open(file_path, 'wb') as file:
            file.write(response.content)
    
        print(f"{file_path} downloaded successfully.")
        
    verb_dataset = folder_path

In [ ]:
# Confirming the model existence
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Model_Reader(model)

In [ ]:
# List all variables
MW.Variable_Search('', model)

In [ ]:
# confirm MW functions work appropriately with model varnames dictionary
MW.Variable_Search('')

In [ ]:
MW.Variable_Search('Phase Space Density', model)

In [ ]:
# Cleaning the existing '_list.txt' and '_times.txt'
import os.path
from os.path import isfile
from os import remove
list_file = os.path.join(verb_dataset, model+'_list.txt') 
times_file = os.path.join(verb_dataset, model+'_times.txt')
if isfile(list_file):
    print(f"{list_file} is found and will be deleted")
    remove(list_file)
if isfile(times_file):
    print(f"{times_file} is found and will be deleted")
    remove(times_file)

In [ ]:
MW.Variable_Search('Phase Space Density', model, verb_dataset)

In [ ]:
MW.Var_3D(model)

In [ ]:
MW.Var_units(model, variables_requested)

In [ ]:
# Confirm time method works with model reader
MW.File_Times(model, verb_dataset)

In [ ]:
# Confirm file list method works with model reader
MW.File_List(model, verb_dataset)

In [ ]:
# Check that time files creation works, that reader works for one variable,
# and that an unknown variable request does not break it.
reader = MW.Model_Reader(model)
kamodo_object = reader(verb_dataset, variables_requested=['Trash'])
kamodo_object

In [ ]:
# Check that reader works for one variable with an unknown variable
kamodo_object = reader(verb_dataset, variables_requested=['Trash', variables_requested[0]])
kamodo_object

In [ ]:
# Test model reader with all variables
kamodo_object = reader(verb_dataset, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Check that reading the time files works and that reader works for one variable,
kamodo_object = reader(verb_dataset, variables_requested=variables_requested[:1])
kamodo_object

In [ ]:
# Verify start time
kamodo_object.filedate

In [ ]:
# Check that reading the time files works, and that the reader works for all variables
kamodo_object = reader(verb_dataset)
kamodo_object

In [ ]:
# Get a list of all of the functionalized variables, both regular and gridded
var_list = list(MW.Variable_Search('', model, verb_dataset, return_dict=True).keys())
varijk_list = sorted(var_list + [item+'_ijk' for item in var_list if 'PSD' in item])
varijk_list

In [ ]:
# Test coordinate range logic for all variables
try:
    MW.Coord_Range(kamodo_object, varijk_list)
except Exception as e:
    print(f'Error during Coord_Range: {e.__class__}, {e}')

In [ ]:
# Does not work if name has "_"
MW.Coord_Range(kamodo_object, ['PSD_lea_ijk'])

In [ ]:
# Check that the kamodo object was built properly.
from math import isnan
xvec = [0., 4., 1., 45]
print(kamodo_object.PSD_lea(xvec))
if isnan(kamodo_object.PSD_lea(xvec)[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')

In [ ]:
# Check that the reader works for the testing subset
kamodo_object = reader(verb_dataset, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Confirm that interpolation works. 
val_psd_lea = kamodo_object.PSD_lea([0., 4., 1., 45])
print(val_psd_lea)
if isnan(val_psd_lea[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
val_psd_lea_int = kamodo_object.PSD_lea_ijk(time=0., L=4., E_e=1., alpha_e=45.)
print(val_psd_lea_int)
if isnan(val_psd_lea_int):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
if not val_psd_lea[0] == val_psd_lea_int:
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')

# This line does not work year
# print(kamodo_object.PSD_lea_ijk(time=1., L=4).shape)

In [ ]:
# Confirm that the interpolator works for each testing variable and type
xvec = [4., 1., 45]
var_list = ['L', 'E_e', 'alpha_e']

for var in var_list:
    print(f'{var}: ')
    kamodo_variable_object = getattr(kamodo_object, var, None)
    if kamodo_variable_object:
        val_kamodo_variable = kamodo_variable_object(xvec)
        print(val_kamodo_variable)
        if isnan(val_kamodo_variable[0]):
            raise AttributeError('Returned value is a NaN.')
        else:
            print('Value is valid.')
    else:
        raise AttributeError(f'{var} is not a kamodo attribute')


In [ ]:
kamodo_object.PSD_lea_ijk(time=0, E_e=1., alpha_e=45.)

In [ ]:
kamodo_object.PSD_lea_ijk(time=2, E_e=1., alpha_e=45.)

In [ ]:
kamodo_object.PSD_lmk_ijk(time=2, mu=700, K=0.11)

In [ ]:
# For plot and validation
from kamodo_ccmc.tools.plotfunctions import toColor, figMods
import numpy as np

# ployly has issue with plotting when LaTeX should be displayed (https://github.com/plotly/plotly.py/issues/4336)
# This disables interactive visualization for IDE like PyCharm, but allows the figure to render correctly.
# If the notebook is executed in the browser, the default render should use 'iframe'.
# Until plotly issue will be fixed this is the best method I was able to find.
import plotly.io as pio
pio.renderers.default = 'iframe+png'

In [ ]:
# Plot L profiles
fig = kamodo_object.plot('PSD_lea_ijk', plot_partial={'PSD_lea_ijk': {'time': 1.5, 'E_e': 1., 'alpha_e' : 45}})
fig.data[0].update(y=np.log10(fig.data[0].y))
fig.update_layout(width=1200)
fig.show()
#fig.update_layout(xaxis={'autorange': True}, yaxis={'autorange': True})

In [ ]:
# Plot L profiles
fig = kamodo_object.plot('PSD_lea_ijk', plot_partial={'PSD_lea_ijk': {'time': 1., 'L': 4.}})
fig.data[0].update(x=np.log10(fig.data[0].y), zauto=False, zmin=-7)
fig.data[0].colorbar.title.side = 'right'
fig.update_layout(width=1200)
figMods(fig, colorscale="Rainbow", log10=True)
fig.show()

In [ ]:
# Variable PSD_lea cannot be plotted by Kamodo 
# fig = kamodo_object.plot('PSD_lea', plot_partial={'PSD_lea': {'time': 1.5, 'E_e': 1., 'alpha_e' : 0.5}})
# fig.data[0].update(y=np.log10(fig.data[0].y))
# fig.update_layout(width=1200)
# fig.show()

In [ ]:
# Plot time dependent PSD slice. Note, that Kamodo need minimum 4 points in time to construct this plot.
fig = kamodo_object.plot('PSD_lea_ijk', plot_partial={'PSD_lea_ijk': {'L': 4., 'E_e': 1., 'alpha_e' : 45.}})
fig.data[0].update(y=np.log10(fig.data[0].y))
fig.update_layout(width=1200)
fig.show()

In [ ]:
# Plot Energy vs alpha 2d plot
fig = kamodo_object.plot('PSD_lea_ijk', plot_partial={'PSD_lea_ijk': {'time': 1.5, 'L': 4.}})
fig.data[0].update(x=np.log10(fig.data[0].x), zauto=False, zmin=-7)
fig.update_layout(width=1200)
fig.data[0].colorbar.title.side = 'right'
figMods(fig, colorscale="Rainbow", log10=True)
fig.show()
#print(fig)
#toColor(fig, colorscale="Viridis")

In [ ]:
# Plot L vs time. Note, Kamodo needs at least 4 points in time to construct this plot 
fig = kamodo_object.plot('PSD_lea_ijk', plot_partial={'PSD_lea_ijk': {'E_e': 1., 'alpha_e' : 45.}})
fig.data[0].update(zauto=False, zmin=-7)
fig.update_layout(width=1200)
fig.data[0].colorbar.title.side = 'right'
# Plotly options for colorscale (https://plotly.com/python/reference/contour/#contour-colorscale)
figMods(fig, colorscale="Rainbow", log10=True)
fig.show()
#toColor(fig, colorscale="Rainbow")

In [ ]:
fig = kamodo_object.plot('flux_lea_ijk', plot_partial={'flux_lea_ijk': {'E_e': 1., 'alpha_e' : 45.}})
fig.data[0].update(zauto=False, zmin=-7)
fig.update_layout(width=1200)
fig.data[0].colorbar.title.side = 'right'
# Plotly options for colorscale (https://plotly.com/python/reference/contour/#contour-colorscale)
figMods(fig, colorscale="Rainbow", log10=True)
fig.show()

In [ ]:
fig = kamodo_object.plot('PSD_lmk_ijk', plot_partial={'PSD_lmk_ijk': {'mu': 700., 'K' : 0.11}})
fig.data[0].update(zauto=False, zmin=-7)
fig.update_layout(width=1200)
fig.data[0].colorbar.title.side = 'right'
# Plotly options for colorscale (https://plotly.com/python/reference/contour/#contour-colorscale)
figMods(fig, colorscale="Rainbow", log10=True)
fig.show()

In [ ]:
# Test that more than one variable works through the flythrough
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt
start_utcts = kamodo_object.filedate.timestamp()
end_utcts = (kamodo_object.filedate + dt.timedelta(hours=1)).timestamp()
results = SF.ModelFlythrough(model, verb_dataset, [variables_requested[0]], [start_utcts, end_utcts], [4., 5.], [1., 1.],
                             [45., 45.], 'LEA-rb')
results[variables_requested[0]]

In [ ]:
# Test that one variable works through the flythrough
results = SF.ModelFlythrough(model, verb_dataset, [variables_requested[0]], [start_utcts], [4.], [1.],
                             [45.], 'LEA-rb')
results[variables_requested[0]]

In [ ]:
# import sys
# sys.exit("Stop here")